In [ ]:
#%reset -f
import sys,os
from pathlib import Path

from ipywidgets import AppLayout, Button, Layout

import ipywidgets as widgets
from ipywidgets import AppLayout
from IPython.display import display, HTML



#notebook1_path = Path(r"C:\Users\sushil\OneDrive - Awan Infotech Private Limited\Documents\jubiter").resolve()
#if str(notebook1_path) not in sys.path:
    #sys.path.append(str(notebook1_path))

#from ipynb.fs.full.azure_functions import get_subscriptions


In [267]:
import nbformat
from IPython.core.interactiveshell import InteractiveShell


notebook_path_azure = r'C:\Users\sushil\OneDrive - Awan Infotech Private Limited\Documents\jubiter\azure_functions.ipynb'
notebook_path_widgets = r'C:\Users\sushil\OneDrive - Awan Infotech Private Limited\Documents\jubiter\widgets.ipynb'

# Load the notebook

with open(notebook_path_azure) as f:
    nb1 = nbformat.read(f, as_version=4)
with open(notebook_path_widgets) as f:
    nb2 = nbformat.read(f, as_version=4)

# Create an interactive shell
shell = InteractiveShell.instance()

# Execute each cell in the notebook
for cell in nb1.cells:
    if cell.cell_type == 'code':
        shell.run_cell(cell.source)
for cell in nb2.cells:
    if cell.cell_type == 'code':
        shell.run_cell(cell.source)

In [ ]:
subscriptions = get_subscriptions()
#print(subscriptions)

In [ ]:
# Env Selector
# Dropdown Selector - Subscription, Resource Group, Resource Type
# Filter Display Box - Subscription, Resource Group, Resource Type

In [ ]:
dropdown_options = list(subscriptions.itertuples(index=False, name=None))
dropdown_subs = widgets.Dropdown(
    options=dropdown_options,
    #value=2,
    description='Number:',
)

output_subs = widgets.Output()
def on_dropdown_change(change):
    with output_subs:
        output_subs.clear_output()
        display(dropdown_subs)
        print(f'Selected Subscription ID: {change["new"]}')

dropdown_subs.observe(on_dropdown_change, names='value')

#display(dropdown_subs,output_subs)

with output_subs:
    display(dropdown_subs)
    print(f'Selected Subscription ID: {dropdown_subs.value}')




In [270]:
resources_df = pd.DataFrame()
for index,subs in subscriptions.iterrows():
    #print(subs['subscription_name'])
    result_df = get_resources(subs['subscription_id'])
    resources_df = pd.concat([resources_df, result_df], ignore_index=True)

#print(resources_df.head(10))

#grouped = resources_df.groupby(['Type']) #'subscription_id',
#print(grouped)

# for resource_type, group in grouped:
#     print(f"Resource Type: {resource_type}")
#     print(group)
#     print("-----")

grouped_df = resources_df.groupby(['Type']).size().reset_index(name='count') #'subscription_id',
resource_summary_df = grouped_df.sort_values(by='count', ascending=False)
#print(resource_summary_df)



In [271]:
def get_filtered_resource_data_in_container(df):
    # resources_df resource_summary_df
    output = widgets.Output()

    def highlight_rows(row):
        return ['background-color: paleturquoise' if row.name % 2 == 0 else 'background-color: peachpuff' for _ in row]
    # lightgreen lightblue

    styled_df = df.style.apply(highlight_rows, axis=1)

    styled_df = styled_df.set_table_styles([
    {'selector': 'thead th', 'props': [('background-color', 'paleturquoise')]},
    {'selector': 'tbody th', 'props': [('background-color', 'paleturquoise')]}
    ])



    # Display the dataframe inside the Output widget
    with output:
        display(styled_df)
        
    scroll_size = 300 if len(df) > 9 else len(df) * 35
    scroll_size += 0 if len(df) > 2 else len(df) + 30
    scrollable_container = widgets.VBox([output])
    scrollable_container.layout = widgets.Layout(overflow='auto', height=f'{scroll_size}px')
    return scrollable_container


In [272]:
resource_title = ()
resource_data = []
for index,item in resource_summary_df.iterrows():
    #print(f"{item['subscription_id']} - {item['Type']} - {item['count']}") {item['subscription_id']} - 
    str = f"{item['Type']} - {item['count']}"
    resource_title = resource_title + (str,)
    filtered_df = resources_df[(resources_df['Type'] == item['Type'] ) ].sort_values(by='subscription_id').reset_index(drop=True) # & (resources_df['B'] < 50)]
    filtered_df.index = filtered_df.index + 1
    resource_data.append(get_filtered_resource_data_in_container(filtered_df)) #widgets.IntSlider())
#print(resource_title)
#print(resource_data)
accordion = widgets.Accordion(children=resource_data, titles=resource_title)

for i, child in enumerate(accordion.children):
    color = 'lightblue' if i % 2 == 0 else 'lightgreen'
    child.layout = widgets.Layout(background_color=color)

accordion_html = """
<style>
    .accordion-header:nth-child(2n) {
        background-color: lightblue !important;
    }
    .accordion-header:nth-child(2n+1) {
        background-color: lightgreen !important;
    }
</style>
"""

# Display the custom CSS
display(HTML(accordion_html))

#display(accordion)


In [ ]:
#create_button('Header', 'success')

In [ ]:
# Result Box
# Create an Output widget with a border
# output_widget = widgets.Output(layout={'border': '1px solid black','padding':'10px',})

# # Fill the Output widget with some content
# with output_widget:
#     print("This is a box with a border")

# widget_list = []
# for i in range(6):
#     widget_list.append(output_widget)

#labels = [widgets.Label(f'Label {i}') for i in range(1, 10)]
#print(labels)

# Create a Box widget to contain the Output widget
box_result = widgets.Box([accordion], layout=widgets.Layout(
    border='1px solid black', 
    padding='10px',
    #margin='10px',
    #width='50%',
    height='500px', #400px',
    background_color='lightblue',
    #display='flex',
    #flex_flow='row wrap',
    flex_flow='column',
    overflow='auto',
    #align_items='center',
    justify_content='space-between'
    ))

# Display the box
#display(box)


In [278]:
toggle_buttons = widgets.ToggleButtons(
    options=['Slow', 'Regular', 'Fast'],
    description='Environment:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    #tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    #icons=['check'] * 3
)
def on_toggle_change(change):
    selected_option = change['new']
    label1.value = f'Selected: {selected_option}'
    #with output:
        #output.clear_output()
        #print(f'Function called with {selected_option}')

# Attach the function to the ToggleButtons widget
toggle_buttons.observe(on_toggle_change, names='value')

In [ ]:
# Create two vertical elements
label1 = widgets.Label('Headers 2 - Label 1')
label2 = widgets.Label('Headers 2 - Label 2')
HBox_dropdown_options = widgets.HBox([label1])
HBox_filter_selections = widgets.HBox([output_subs]) 
#box_subs = widgets.HBox([output_subs]) 
items = [create_button(f"abc * {i}", 'success') for i in range(1,10)]
HBox_subfilter = widgets.HBox(items) #[widgets.Label('VBox 1 - Label 1'), widgets.Label('VBox 1 - Label 2')])
HBox_ = widgets.HBox([widgets.Label('VBox 2 - Label 1'), widgets.Label('VBox 2 - Label 2')])

# Arrange the two vertical elements in a horizontal box layout
vbox = widgets.VBox([toggle_buttons,
                     HBox_dropdown_options,
                     HBox_filter_selections,
                     #box_subs,
                     HBox_subfilter,
                     HBox_,
                     box_result])

# Display the layout
#display(vbox)


In [281]:
header = widgets.HTML("<h2>Header</h2>")
left_sidebar = widgets.HTML("<p>Left Sidebar</p>")
main_content = widgets.HTML("<p>Main Content</p>")
right_sidebar = widgets.HTML("<p>Right Sidebar</p>")
footer = widgets.HTML("<p>Footer</p>")

# Apply background colors using the layout attribute
header.layout = widgets.Layout(background_color='#f8f9fa')
left_sidebar.layout = widgets.Layout(background_color='#e9ecef')
main_content.layout = widgets.Layout(background_color='#ffffff')
right_sidebar.layout = widgets.Layout(background_color='#e9ecef')
footer.layout = widgets.Layout(background_color='#f8f9fa')



In [287]:
# Create the AppLayout
app_layout = widgets.AppLayout(
    header=None, #header,
    left_sidebar=left_sidebar,
    center=vbox,
    right_sidebar=right_sidebar,
    footer=footer,
    pane_widths=[1, 8, 1],
    pane_heights=[1, 8, 1]
)

# Display the AppLayout
display(app_layout)



AppLayout(children=(HTML(value='<p>Footer</p>', layout=Layout(grid_area='footer')), HTML(value='<p>Left Sideba…

In [ ]:
custom_css = """
<style>
    .widget-app-layout {
        width: 100vw;
        height: 100vh;
        display: flex;
        flex-direction: column;
    }
    .widget-app-layout > .widget-box {
        flex: 1;
    }
    .widget-app-layout > .widget-box:nth-child(1) {
        background-color: #f8f9fa;
    }
    .widget-app-layout > .widget-box:nth-child(2) {
        display: flex;
    }
    .widget-app-layout > .widget-box:nth-child(2) > .widget-box:nth-child(1) {
        background-color: #e9ecef;
        flex: 1;
    }
    .widget-app-layout > .widget-box:nth-child(2) > .widget-box:nth-child(2) {
        background-color: #ffffff;
        flex: 2;
    }
    .widget-app-layout > .widget-box:nth-child(2) > .widget-box:nth-child(3) {
        background-color: #e9ecef;
        flex: 1;
    }
    .widget-app-layout > .widget-box:nth-child(3) {
        background-color: #f8f9fa;
    }
</style>
"""


In [ ]:
# # Display the custom CSS
# display(HTML(custom_css))

# # Display the AppLayout
# display(app_layout)


In [ ]:
# AppLayout(header=create_button('Header', 'success'),
#           left_sidebar=create_button('Left', 'info'),
#           center=vbox, #center_button,
#           right_sidebar=create_button('Right', 'info'),
#           footer=create_button('Footer', 'success'),
#           pane_widths=[1, 8.5, 0.5],
#           pane_heights=[0.5, 5, '30px'])

In [ ]:
# out = widgets.Output(layout={'border': '1px solid black'})
# out

In [ ]:

# out.clear_output()
# with out:
#     for i in range(10):
#         print(i, 'Hello world!')

In [ ]:
# accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text()], titles=('Slider', 'Text'))
# accordion

In [ ]:
# items = [widgets.Label(str(i)) for i in range(8)]
# widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 100px)"))